In [1]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms

In [2]:
# Device configuration
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [3]:
# Hyper parameters
num_epochs = 5
num_classes = 10
batch_size = 100
learning_rate = 0.001

In [4]:
# MNIST dataset
train_dataset = torchvision.datasets.MNIST(root='../../data/',
                                           train=True,
                                           transform=transforms.ToTensor(),
                                           download=True)

test_dataset = torchvision.datasets.MNIST(root='../../data/',
                                          train=False,
                                          transform=transforms.ToTensor())

100%|██████████| 9.91M/9.91M [00:00<00:00, 18.6MB/s]
100%|██████████| 28.9k/28.9k [00:00<00:00, 489kB/s]
100%|██████████| 1.65M/1.65M [00:00<00:00, 4.01MB/s]
100%|██████████| 4.54k/4.54k [00:00<00:00, 5.87MB/s]


In [5]:
# Data loader
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=batch_size,
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=batch_size,
                                          shuffle=False)

In [7]:
# Convolutional neural network (two convolutional layers)
class ConvNet(nn.Module):
    def __init__(self, num_classes=10):
        super(ConvNet, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(1, 16, kernel_size=5, stride=1, padding=2),
            nn.BatchNorm2d(16),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
        self.layer2 = nn.Sequential(
            nn.Conv2d(16, 32, kernel_size=5, stride=1, padding=2),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
        self.fc = nn.Linear(7*7*32, num_classes)

    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.reshape(out.size(0), -1)
        out = self.fc(out)
        return out

In [12]:
import torch
import torch.nn as nn

class DeeperConvNet(nn.Module):
    def __init__(self, num_classes=10):
        super(DeeperConvNet, self).__init__()
        # First convolutional block
        self.layer1 = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=5, stride=1, padding=2),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )

        # Second convolutional block
        self.layer2 = nn.Sequential(
            nn.Conv2d(32, 64, kernel_size=5, stride=1, padding=2),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )

        # Third convolutional block
        self.layer3 = nn.Sequential(
            nn.Conv2d(64, 128, kernel_size=5, stride=1, padding=2),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )

        # Fully connected layers
        # The input size for the first fully connected layer needs to be calculated
        # based on the output of the last convolutional layer.
        self.fc1 = nn.Linear(3*3*128, 512)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(512, num_classes)

    def forward(self, x):
        # Pass input through the convolutional layers
        out = self.layer1(x)
        out = self.layer2(out)
        out = self.layer3(out)

        # Flatten the output for the fully connected layers
        out = out.reshape(out.size(0), -1)

        # Pass through the fully connected layers
        out = self.fc1(out)
        out = self.relu(out)
        out = self.fc2(out)

        return out

In [13]:
model = DeeperConvNet(num_classes).to(device)

In [14]:
# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [15]:
# Train the model
total_step = len(train_loader)
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        images = images.to(device)
        labels = labels.to(device)

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if (i+1) % 100 == 0:
            print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}'
                   .format(epoch+1, num_epochs, i+1, total_step, loss.item()))


Epoch [1/5], Step [100/600], Loss: 0.2500
Epoch [1/5], Step [200/600], Loss: 0.0859
Epoch [1/5], Step [300/600], Loss: 0.0554
Epoch [1/5], Step [400/600], Loss: 0.0766
Epoch [1/5], Step [500/600], Loss: 0.0237
Epoch [1/5], Step [600/600], Loss: 0.0131
Epoch [2/5], Step [100/600], Loss: 0.1258
Epoch [2/5], Step [200/600], Loss: 0.0125
Epoch [2/5], Step [300/600], Loss: 0.0570
Epoch [2/5], Step [400/600], Loss: 0.0027
Epoch [2/5], Step [500/600], Loss: 0.0339
Epoch [2/5], Step [600/600], Loss: 0.0069
Epoch [3/5], Step [100/600], Loss: 0.0414
Epoch [3/5], Step [200/600], Loss: 0.0468
Epoch [3/5], Step [300/600], Loss: 0.1681
Epoch [3/5], Step [400/600], Loss: 0.0055
Epoch [3/5], Step [500/600], Loss: 0.0203
Epoch [3/5], Step [600/600], Loss: 0.0110
Epoch [4/5], Step [100/600], Loss: 0.0099
Epoch [4/5], Step [200/600], Loss: 0.0094
Epoch [4/5], Step [300/600], Loss: 0.0644
Epoch [4/5], Step [400/600], Loss: 0.0141
Epoch [4/5], Step [500/600], Loss: 0.0102
Epoch [4/5], Step [600/600], Loss:

In [16]:
# Test the model
model.eval()  # eval mode (batchnorm uses moving mean/variance instead of mini-batch mean/variance)
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print('Test Accuracy of the model on the 10000 test images: {} %'.format(100 * correct / total))

# Save the model checkpoint
torch.save(model.state_dict(), 'model.ckpt')

Test Accuracy of the model on the 10000 test images: 99.14 %
